In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam, RMSprop, Adagrad

# 데이터셋 설정
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_set = torchvision.datasets.FashionMNIST(root='./data',
                                              train=True,
                                              download=True,
                                              transform=transform)
test_set = torchvision.datasets.FashionMNIST(root='./data',
                                             train=False,
                                             download=True,
                                             transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28, 256)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(256, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output(x)
        return x

model = MLP()

# 손실 함수 및 최적화 설정
loss_fn = nn.CrossEntropyLoss()
optimizers = {
    'SGD': SGD(model.parameters(), lr=0.1),
    'Momentum': SGD(model.parameters(), lr=0.1, momentum=0.9),
    'Adagrad': Adagrad(model.parameters(), lr=0.1),
    'RMSprop': RMSprop(model.parameters(), lr=0.1),
    'Adam': Adam(model.parameters(), lr=0.1)
}

# 훈련 루프
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 평가
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 훈련 및 평가 실행
for name, optimizer in optimizers.items():
    print(f"Training with optimizer: {name}")
    train(train_loader, model, loss_fn, optimizer)
    print(f"Testing with optimizer: {name}")
    test(test_loader, model)


100%|██████████| 26421880/26421880 [00:01<00:00, 14768250.79it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272265.76it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5068988.03it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3269080.54it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Training with optimizer: SGD
loss: 2.323056  [    0/60000]
loss: 0.587839  [ 6400/60000]
loss: 0.721406  [12800/60000]
loss: 0.771263  [19200/60000]
loss: 0.643121  [25600/60000]
loss: 0.389304  [32000/60000]
loss: 0.485570  [38400/60000]
loss: 0.502587  [44800/60000]
loss: 0.505835  [51200/60000]
loss: 0.462324  [57600/60000]
Testing with optimizer: SGD
Test Error: 
 Accuracy: 81.8%, Avg loss: 0.509029 

Training with optimizer: Momentum
loss: 0.582544  [    0/60000]
loss: 0.645904  [ 6400/60000]
loss: 0.568057  [12800/60000]
loss: 0.455921  [19200/60000]
loss: 0.476197  [25600/60000]
loss: 0.385763  [32000/60000]
loss: 0.590658  [38400/60000]
loss: 0.678371  [44800/60000]
loss: 0.705283  [51200/60000]
loss: 0.549218  [57600/60000]
Testing with optimizer: Momentum
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.621135 

Training with optimizer: Adagrad
loss: 0.728879  [    0/60000]
loss: 1.062733  [ 64